In [1]:
# Inicializa una lista para almacenar la información de los jugadores
player_data = {}

In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Inicializa el WebDriver
driver = webdriver.Chrome()

# Navega a la página de estadísticas
url = 'https://fantasy.premierleague.com/statistics'
driver.get(url)

# Espera a que la ventana emergente de cookies esté visible y haz clic en "Accept All Cookies"
try:
    accept_cookies_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'onetrust-accept-btn-handler'))
    )
    accept_cookies_button.click()
except Exception as e:
    print(f"No se pudo aceptar las cookies: {e}")

# Espera a que la página cargue completamente
time.sleep(5)

# Función para extraer datos de jugadores en la página actual
def scrape_page():
    tbody = driver.find_element(By.TAG_NAME, 'tbody')
    rows = tbody.find_elements(By.TAG_NAME, 'tr')

    for row in rows:
        player_button = row.find_element(By.CLASS_NAME, 'ElementInTable__Name-sc-y9xi40-1')
        player_name = player_button.text
        driver.execute_script("arguments[0].click();", player_button)
        time.sleep(2)
        
        dialog = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'root-dialog')))
        lines = dialog.text.split('\n')

        anuncio = None
        if lines[1] not in ['Goalkeeper', 'Defender', 'Midfielder', 'Forward']:
            anuncio = lines[1]
            del lines[1]

        player_name=lines[2]

        # Extrae los datos principales
        dict = {
            'Anuncio': anuncio,
            'posicion': lines[1],
            'nombre_completo': lines[2],
            'equipo': lines[3],
            'precio': lines[5],
            'Form': lines[8],
            'Pts / Match': lines[10],
            'GW8 Pts': lines[13],
            'Total Pts': lines[15],
            'Total Bonus': lines[17],
            'ICT Index': lines[19],
            'TSB%': lines[22],
            'info': lines[1],
        }

        # Procesar estadísticas adicionales
        for i in lines:
            if '  Totals   ' in i:
                break

        campos = ['Pts', 'ST', 'MP', 'GS', 'A', 'xG', 'xA', 'xGI', 'CS', 'GC', 'xGC', 'OG', 'PS', 'PM', 'YC', 'RC', 'S', 'BP', 'BPS', 'I', 'C', 'T', 'II', 'NT', 'SB', '£']
        datos = i.replace('  Totals   ', '').replace('-', '').replace(' ', ',').strip()
        datos_list = [float(x) if x else None for x in datos.split(',')]

        dict2 = {}
        for i in range(0, len(campos) - 1, 1):
            dict2[campos[i]] = datos_list[i]

        dict['info'] = dict2
        player_data[player_name] = dict

        try:
            # Cierra el diálogo del jugador
            close_button = WebDriverWait(driver, 15).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'div.Dialog__CloseButtonWrap-sc-5bogmv-0 button.Dialog__CloseButton-sc-5bogmv-1'))
            )
            close_button.click()
        except Exception as e:
            print(f"No se pudo cerrar el diálogo para {player_name}: {e}")

# Función para verificar si existe el botón "Next" y avanzar de página
def next_page():
    # siguiente pagina
    try:
        # Espera hasta que el botón Next esté presente y sea clickeable
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[./span[text()='Next']]"))
        )    
        next_button.click()
        return True
    except:
        return False
    
#adelantar paginas    
for i in range(1,14,1):
    next_page()    

##Recorre todas las páginas de jugadores
pagina = 1
while True:
    print(f'Página: {pagina}')
    scrape_page()
    pagina = pagina + 1
    if not next_page():
        break

# Cierra el navegador
#driver.quit()



Página: 1
Página: 2
Página: 3
Página: 4
Página: 5
Página: 6
Página: 7
Página: 8
Página: 9
Página: 10


In [7]:
#guardar diccionario
import json
# Guardar el diccionario en un archivo JSON
with open('player_data.json', 'w') as arch_salida:
    json.dump(player_data, arch_salida)

In [8]:
# Cargar el diccionario desde un archivo JSON
import json
with open('player_data.json', 'r') as arch_entrada:
    diccionario_cargado = json.load(arch_entrada)
len(diccionario_cargado)

667

In [ ]:
# INSERTAR EN BASE DE DATOS

import psycopg2

# Función para convertir el precio y TSB (porcentaje) a float
def convertir_precio(precio_str):
    # Eliminar el símbolo de moneda (£) y 'm', y convertir a float
    return float(precio_str.replace('£', '').replace('m', '').replace(' ', ''))

def convertir_porcentaje(porcentaje_str):
    # Eliminar el símbolo de porcentaje y convertir a float
    return float(porcentaje_str.replace('%', ''))

# Configuración de conexión
connection = psycopg2.connect(
host = "localhost",
database = "bd_aprendizaje",
user = "soporte",
password = ""
)

# Crear un cursor
cursor = connection.cursor()

# Consulta SQL para insertar datos en la tabla
insert_query = """
    INSERT INTO jugadores (
        nombre, anuncio, posicion, nombre_completo, equipo, precio, Form, Pts_Match, GW8Pts, TotalPts, 
        TotalBonus, ICTIndex, TSB, Pts, ST, MP, GS, A, xG, xA, xGI, CS, GC, xGC, OG, PS, 
        PM, YC, RC, S, BP, BPS, I, C, T, II
    ) VALUES (%s,%s, %s,%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# Recorrer cada jugador en el diccionario
for key, jugador in diccionario_cargado.items():
    print(key)
    precio = convertir_precio(jugador['precio'])
    TSB = convertir_porcentaje(jugador['TSB%'])
    
    datos_a_insertar = (
    key,    
    diccionario_cargado[key]['Anuncio'],
    diccionario_cargado[key]['posicion'], 
    diccionario_cargado[key]['nombre_completo'], 
    diccionario_cargado[key]['equipo'], 
    precio, 
    float(diccionario_cargado[key]['Form']), 
    float(diccionario_cargado[key]['Pts / Match']), 
    float(diccionario_cargado[key]['GW8 Pts']), 
    float(diccionario_cargado[key]['Total Pts']), 
    float(diccionario_cargado[key]['Total Bonus']), 
    float(diccionario_cargado[key]['ICT Index']), 
    TSB, 
    diccionario_cargado[key]['info']['Pts'], diccionario_cargado[key]['info']['ST'], diccionario_cargado[key]['info']['MP'], diccionario_cargado[key]['info']['GS'], diccionario_cargado[key]['info']['A'], 
    diccionario_cargado[key]['info']['xG'], diccionario_cargado[key]['info']['xA'], diccionario_cargado[key]['info']['xGI'], diccionario_cargado[key]['info']['CS'], diccionario_cargado[key]['info']['GC'], 
    diccionario_cargado[key]['info']['xGC'], diccionario_cargado[key]['info']['OG'], diccionario_cargado[key]['info']['PS'], diccionario_cargado[key]['info']['PM'], diccionario_cargado[key]['info']['YC'], 
    diccionario_cargado[key]['info']['RC'], diccionario_cargado[key]['info']['S'], diccionario_cargado[key]['info']['BP'], diccionario_cargado[key]['info']['BPS'], diccionario_cargado[key]['info']['I'], 
    diccionario_cargado[key]['info']['C'], diccionario_cargado[key]['info']['T'], diccionario_cargado[key]['info']['II']
    )
    # Ejecutar la consulta de inserción
    cursor.execute(insert_query, datos_a_insertar)

# Confirmar los cambios
connection.commit()

# Cerrar la conexión
cursor.close()
connection.close()

In [ ]:
'''
# query para elegir jugadores
select * from jugadores
where 1=1
and lower(posicion) like '%mid%'
and anuncio is null
and precio <= 6.2
order by pts_match desc, form desc, st desc, precio asc
'''